# PyROOT Rig Binner
Sebastiaan Venendaal
<ul>
    <li>Created on 02-11-20</li>
    <li>Last updated on 02-11-20</li>
</ul>

In [16]:
import numpy as np
import ROOT
%jsroot on

## Data

In [17]:
# Include header file
ROOT.gInterpreter.Declare('#include "../Ntp.h"')

# Make a Tree Chain
chain = ROOT.TChain('Simp')
# Fill Chain with all available Runs
chain.Add('../../Simp.root')
    
# Print number of entries
chain_events = chain.GetEntries()
print('Data Entries in TChain:', chain_events)

# Visual check
#chain.Print()

Data Entries in TChain: 889410


## Cuts

In [39]:
# CUT CONFIG
lay_crit = '>=' # criterion symbol for bad charge status in tracker layers
geo_crit = '1.2' # criterion strength of geo-magnetic cut-off (standard convention is 1.2)

# CUTS
# General
CGa = ROOT.TCut('tof_beta > 0') # down-going particles
CGb = ROOT.TCut('trk_rig > 0 && trk_rig <= 22') # deuteron identification range
CGc = ROOT.TCut('rich_beta > 0.95 && rich_beta < 0.98') # small beta slice (optional)
CG = CGa + CGb
# Quality
CQa = ROOT.TCut('trk_chisqn[0] < 10 && trk_chisqn[1] < 10 && trk_chisqn[0] > 0 && trk_chisqn[1] > 0') # well-reconstructed particles
CQb = ROOT.TCut('status % 10 == 1') # single particle events
CQ = CQa + CQb
# Charge
CZa = ROOT.TCut('trk_q_inn > 0.80 && trk_q_inn < 1.30') # single charge particles
CZb = ROOT.TCut('trk_q_lay[4]{0}0 && trk_q_lay[1]{0}0 && trk_q_lay[2]{0}0 && trk_q_lay[3]{0}0 &&'.format(lay_crit) +
                'trk_q_lay[5]{0}0 && trk_q_lay[6]{0}0 && trk_q_lay[7]{0}0 && trk_q_lay[8]{0}0 &&'.format(lay_crit) +
                'trk_q_lay[0]{0}0'.format(lay_crit)) # no bad charge status throughout tracker
CZ = CZa + CZb
# Geo-magnetic cut-off
CR = ROOT.TCut('trk_rig > {0}*cf'.format(geo_crit)) # geo-magnetic cut-off
# Consistency
CC = ROOT.TCut('abs(tof_beta-rich_beta)/tof_beta < 0.05') # TOF - RICH beta consistency

Call = CG + CQ + CZ + CR + CC
#Call = ROOT.TCut('')


## Rigidity Slices

In [40]:
slices = np.linspace(0, 22, 9) # 9 values for 8 bins/slices
print(slices)

rig_cuts = [ROOT.TCut('trk_rig > {} && trk_rig < {}'.format(slices[i], slices[i+1])) for i in range(len(slices)-1)]

print()
for cut in rig_cuts:
    print(cut)

[ 0.    2.75  5.5   8.25 11.   13.75 16.5  19.25 22.  ]

trk_rig > 0.0 && trk_rig < 2.75
trk_rig > 2.75 && trk_rig < 5.5
trk_rig > 5.5 && trk_rig < 8.25
trk_rig > 8.25 && trk_rig < 11.0
trk_rig > 11.0 && trk_rig < 13.75
trk_rig > 13.75 && trk_rig < 16.5
trk_rig > 16.5 && trk_rig < 19.25
trk_rig > 19.25 && trk_rig < 22.0


In [62]:
par = 'trk_rig'
bins = 50
xrange = '0, 22'

canvas = ROOT.TCanvas("trial","trial", 800, 500)
chain.Draw('{} >> hist1({}, {})'.format(par, bins, xrange), Call+rig_cuts[0], '')
chain.Draw('{} >> hist2({}, {})'.format(par, bins, xrange), Call+rig_cuts[1], 'SAME')
chain.Draw('{} >> hist3({}, {})'.format(par, bins, xrange), Call+rig_cuts[2], 'SAME')
chain.Draw('{} >> hist4({}, {})'.format(par, bins, xrange), Call+rig_cuts[3], 'SAME')
chain.Draw('{} >> hist5({}, {})'.format(par, bins, xrange), Call+rig_cuts[4], 'SAME')
chain.Draw('{} >> hist6({}, {})'.format(par, bins, xrange), Call+rig_cuts[5], 'SAME')
chain.Draw('{} >> hist7({}, {})'.format(par, bins, xrange), Call+rig_cuts[6], 'SAME')
chain.Draw('{} >> hist8({}, {})'.format(par, bins, xrange), Call+rig_cuts[7], 'SAME')

ROOT.hist1.SetLineColor(ROOT.kGray)
ROOT.hist2.SetLineColor(ROOT.kRed)
ROOT.hist3.SetLineColor(ROOT.kOrange)
ROOT.hist4.SetLineColor(ROOT.kGreen)
ROOT.hist5.SetLineColor(ROOT.kCyan)
ROOT.hist6.SetLineColor(ROOT.kBlue)
ROOT.hist7.SetLineColor(ROOT.kMagenta)
ROOT.hist8.SetLineColor(ROOT.kBlack)

ROOT.hist1.SetAxisRange(0, 1500, "Y")

canvas.Draw()

he1 = ROOT.hist1.GetEntries()
he2 = ROOT.hist2.GetEntries()
he3 = ROOT.hist3.GetEntries()
he4 = ROOT.hist4.GetEntries()
he5 = ROOT.hist5.GetEntries()
he6 = ROOT.hist6.GetEntries()
he7 = ROOT.hist7.GetEntries()
he8 = ROOT.hist8.GetEntries()
total = he1+he2+he3+he4+he5+he6+he7+he8
print('Slice 1: ', int(he1), '({:.1f} %)'.format(he1/total*100))
print('Slice 2: ', int(he2), '({:.1f} %)'.format(he2/total*100))
print('Slice 3: ', int(he3), '({:.1f} %)'.format(he3/total*100))
print('Slice 4: ', int(he4), '({:.1f} %)'.format(he4/total*100))
print('Slice 5: ', int(he5), '({:.1f} %)'.format(he5/total*100))
print('Slice 6: ', int(he6), '({:.1f} %)'.format(he6/total*100))
print('Slice 7: ', int(he7), '({:.1f} %)'.format(he7/total*100))
print('Slice 8: ', int(he8), '({:.1f} %)'.format(he8/total*100))

Slice 1:  0 (0.0 %)
Slice 2:  978 (3.8 %)
Slice 3:  6885 (27.1 %)
Slice 4:  6159 (24.2 %)
Slice 5:  4339 (17.1 %)
Slice 6:  3108 (12.2 %)
Slice 7:  2197 (8.6 %)
Slice 8:  1755 (6.9 %)


Warning in <TCanvas::Constructor>: Deleting canvas with same name: trial


-----
END OF NOTEBOOK